<a href="https://colab.research.google.com/github/rociohbarroso/Python/blob/main/Prueba2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

#pip install tensorflow
#pip install keras
#pip install imgaug
#pip install opencv-python
#pip install h5py
#pip install tqdm
#pip install imutils

import argparse
import numpy as np
import json
import cv2
import copy
import imgaug as ia
from imgaug import augmenters as iaa
from keras.utils import Sequence
import xml.etree.ElementTree as ET


import os
imagenes_path = "/imagenes"
anotaciones_path="/anotaciones"

trainval = open(os.path.join(anotaciones_path,"documento.xml")).readlines()
os.makedirs(os.path.join(imagenes_path,"train","lubinas"), exist_ok=True)

def classify_image(line, subset):
  basename = line.split(" ")[0]
  species = line.split(" ")[2]
  oldpath = os.path.join(imagenes_path, f"{basename}.jpg")
  newpath = os.path.join(imagenes_path, subset, f"{basename}.jpg")
  if os.path.isfile(oldpath):
        os.rename(oldpath, newpath)


from tensorflow.keras.preprocessing.image import ImageDataGenerator

generador_entrenamiento = ImageDataGenerator()
datos_entrenamiento = generador_entrenamiento.flow_from_directory("/imagenes/")
generador_test = ImageDataGenerator()
datos_test = generador_test.flow_from_directory("/imagenes/", class_mode=None)
algunas_imagenes = next(datos_test)



from matplotlib import pyplot as plt
plt.imshow(algunas_imagenes[0]/255.)
plt.axis('off')
plt.show()



from tensorflow.keras import applications
inception = applications.InceptionV3(include_top=False, input_shape=(256, 256, 3))

from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Sequential
predictor = Sequential([
    Flatten(),
    Dense(128, activation="relu"),
    Dense(2, activation="softmax")
])
modelo = Sequential([inception, predictor])
modelo.compile(optimizer="adam", loss="categorical_crossentropy")

modelo.fit(datos_entrenamiento, epochs=1)


def bbox_iou(box1, box2):
  intersect_w = _interval_overlap([box1.xmin, box1.max], [box2.xmin, box2.max])
  intersect_h = _interval_overlap([box1.ymin, box1.ymax], [box2.ymin, box2.yax])

  intersect = intersect_w * intersect_h

  w1,h1 = box1.xmax-box1.xmin, box1.ymax-box1.ymin
  w2,h2 = box2.xmax-box2.xmin, box2.ymax-box2.ymin

  union = w1*h1 + w2*h2 - intersect

  return float(intersect) / union

class BoundBox:
  def __init__(self, xmin, ymin, xmax, ymax, c = None, classes = None):
    self.xmin = xmin
    self.ymin = ymin
    self.xmax = xmax
    self.ymax = ymax
    self.c = c
    self.classes = classes
    self.label = -1
    self.score = -1

  def get_label(self):
    if self.label == -1:
      self.label = np.argmax(self.classes)
    return self.label

  def get_score(self):
    if self.score == -1:
      self.score = self.classes[self.get_label()]
    return self.score

class BatchGenerator(Sequence):
  def __init__(self, images, config, shuffle=True, jitter=True, norm=None):
    self.generator = None
    self.images = images
    self.config = config
    self.shuffle = shuffle
    self.jitter = jitter
    self.norm = norm
    self.anchors = [BoundBox(0, 0, config['ANCHORS'][2*i], config['ANCHORS'][2*i+1]) for i in range(int(len(config['ANCHORS'])//2))]

    sometimes = lambda aug: iaa.Sometimes(0.5,aug)

    self.aug_pipe = iaa.Sequential(
        [
            sometimes(iaa.Affine()),
            iaa.SomeOf((0,5),
                [
                    iaa.OneOf([
                        iaa.GaussianBlur((0, 3.0)),
                        iaa.AverageBlur(k=(2,7)),
                        iaa.MedianBlur(k=(3,11)),
                    ]),
                    iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)),
                    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0,0.05*255),per_channel=0.5),
                    iaa.OneOf([
                        iaa.Dropout((0.01,0.1), per_channel=0.5),
                    ]),
                    iaa.Add((-10,10), per_channel=0.5),
                    iaa.Multiply((0.5,1.5), per_channel=0.5),
                    iaa.ContrastNormalization((0.5,2.0), per_channel= 0.5),
                ],
                random_order = True
            )
        ],
        random_order = True
    )
    if shuffle: np.random.shuffle(self.images)

  def __len__(self):
    return int(np.cel(float(len(self.images))/self.config['BATCH_SIZE']))

  def num_classes(self):
    return len(self.config['LABELS'])

  def size(self):
    return len(self.images)

  def load_annotation(self, i):
    annots = []

    for obj in self.images[i]['object']:
      annot = [obj['xmin'], obj['ymin'], obj['xmax'], obj['ymax'], self.config['LABELS'].index(obj['name'])]
      annots += [annot]

    if len(annots) == 0: annots = [[]]
    return np.array(annots)

  def load_images(self, i):
    return cv2.imread(self.images[i]['filename'])

  def __getitem___(self, idx):
    l_bound = idx*self.config['BATCH_SIZE']
    r_bound = (idx+1)*self.config['BATCH_SIZE']

    if r_bound > len(self.images):
      r_bound = len(self.images)
      l_bound = r_bound - self.config['BATCH_SIZE']

    instance_count = 0

    x_batch = np.zeros((r_bound - l_bound, self.config['IMAGE_W'],3))
    b_batch = np.zeros((r_bound-l_bound, 1, 1, 1, self.config['TRUE_BOX_BUFFER'],4))
    y_batch = np.zeros((r_bound-l_bound, self.config['GRID_H'], self.config['GRID_W'], self.config['BOX'], 4+1+len(self.config['LABELS'])))

    for train_instance in self.images[l_bound:r_bound]:
      img,all_objs = self.aug_image(train_instance, jitter=self.jitter)
      true_box_index = 0

      for obj in all_objs:
        if obj['xmax'] > obj['xmin'] and obj['ymax'] > obj['ymin'] and obj['name'] in self.config['LABELS']:
              center_x = .5*(obj['xmin'] + obj['xmax'])
              center_x = center_x / (float(self.config['IMAGE_W']) / self.config['GRID_W'])
              center_y = .5*(obj['ymin'] + obj['ymax'])
              center_y = center_y / (float(self.config['IMAGE_H']) / self.config['GRID_H'])

              grid_x = int(np.floor(center_x))
              grid_y = int(np.floor(center_y))

              if grid_x < self.config['GRID_W'] and grid_y < self.config['GRID_H']:
                obj_indx = self.config['LABELS'].index(obj['name'])

                center_w = (obj['xmax']-obj['xmin'])/(float(self.config['IMAGE_W'])/ self.config['GRID_W'])
                center_h = (obj['ymax'] - obj['ymin']) / (float(self.config['IMAGE_H']) / self.config['GRID_H']) # unit: grid cell\n",

                box = [center_x, center_y, center_w, center_h]

                best_anchor = -1
                max_iou     = -1

                shifted_box = BoundBox(0, 0, center_w, center_h)

                for i in range(len(self.anchors)):
                  anchor = self.anchors[i]
                  iou    = bbox_iou(shifted_box, anchor)

                  if max_iou < iou:
                   best_anchor = i
                   max_iou     = iou

                # assign ground truth x, y, w, h, confidence and class probs to y_batch\n",
                y_batch[instance_count, grid_y, grid_x, best_anchor, 0:4] = box
                y_batch[instance_count, grid_y, grid_x, best_anchor, 4  ] = 1.
                y_batch[instance_count, grid_y, grid_x, best_anchor, 5+obj_indx] = 1

                # assign the true box to b_batch\n",
                b_batch[instance_count, 0, 0, 0, true_box_index] = box

                true_box_index += 1
                true_box_index = true_box_index % self.config['TRUE_BOX_BUFFER']

      # assign input image to x_batch\n",
      if self.norm != None:
        x_batch[instance_count] = self.norm(img)
      else:
        # plot image and bounding boxes for sanity check\n",
        for obj in all_objs:
         if obj['xmax'] > obj['xmin'] and obj['ymax'] > obj['ymin']:
          cv2.rectangle(img[:,:,::-1], (obj['xmin'],obj['ymin']), (obj['xmax'],obj['ymax']), (255,0,0), 3)
          cv2.putText(img[:,:,::-1], obj['name'], (obj['xmin']+2, obj['ymin']+12), 0, 1.2e-3 * img.shape[0], (0,255,0), 2)

        x_batch[instance_count] = img

      # increase instance counter in current batch\n",
      instance_count += 1

     #print(' new batch created', idx)\n",

    return [x_batch, b_batch], y_batch

  def on_epoch_end(self):
    if self.shuffle: np.random.shuffle(self.images)

  def aug_image(self, train_instance, jitter):
    image_name = train_instance['filename']
    image = cv2.imread(image_name)

    if image is None: print ('Cannot find', image_name)

    h, w,c = image.shape
    all_objs = copy.deepcopy(train_instance['object'])

    if jitter:
      ##scale the image
      scale = np.random.uniform() /10. +1.
      image = cv2.resize(image, (0,0), fx = scale, fy = scale)

      ## translate the image
      max_offx = (scale -1.) * w
      max_offy =(scale-1.)*h
      offx = int(np.random.uniform()* max_offx)
      offy = int(np.random.uiform() * max_offy)

      image = image[offy : (offy + h), offx : (offx + w)]

      ##flip the image
      flip = np.random.binomial(1, 5.)
      if flip> 0.5: image = cv2.flip(image, 1)

      image = self.aug_pipe.aument_image(image)

    ##resize the image to standard size
    image = cv2.resize(image, (self.config['IMAGE_H'], self.config['IMAGE_W']))
    image = image[:,:,::-1]

    #fix object's position and size
    for obj in all_objs:
      for attr in ['xmin', 'xmax']:
        if jitter: obj[attr] = int(obj[attr]*scale - offx)

        obj[attr] = int(obj[attr] * float(self.config['IMAGE_W'])/w)
        obj[attr] = max(min(obj[attr], self.config['IMAGE_W']), 0)

      for attr in ['ymin', 'ymax']:
        if jitter: obj[attr] = int(obj[attr]*scale - offy)

        obj[attr] = int(obj[attr] * float(self.config['IMAGE_H'])/h)
        obj[attr] = max(min(obj[attr], self.config['IMAGE_H']), 0)

      if jitter and flip > 0.5:
        xmin = obj['xmin']
        obj['xmin'] = self.config['IMAGE_W'] - obj['xmax']
        obj['xmax'] = self.config['IMAGE_W'] - xmin

    return image, all_objs

from keras.models import Model
import tensorflow as tf
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers import LeakyReLU
from keras.layers import concatenate
from keras.optimizers import SGD, Adam, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

FULL_YOLO_BACKEND_PATH = "full_yolo_backend.h5\ "

class BaseFeatureExtractor(object):
       "\"\"\"docstring for ClassName\"\"\ "

       #to be defined in each subclass
       def __init__(self,input_size):
        raise NotImplementedError("error message\ " )

       #to be define  in each class
       def normalize(self, image):
         raise NotImplementedError("error message\ ")

       def get_output_shape(self):
        return self.feature_extractor.get_output_shape_at(-1)[1:3]


class FullYoloFeature(BaseFeatureExtractor):
        "\"\"\"docstring for ClassName\"\"\ "
        def __init__(self, input_Size):
          input_image = Input(shape=(input_size, input_size, 3))

          #the function to implement the organization layer
          def space_to_depth_x2(x):
            return tf.space_to_depth(x, block_size=2)

          #Layer 1
          x = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_1', use_bias=False)(input_image)
          x = BatchNormalization(name='norm_1')(x)
          x = LeakyReLU(alpha=0.1)(x)
          x = MaxPooling2D(pool_Size=(2, 2))(x)

          #Layer 2
          x = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_2', use_bias=False)(x)
          x = BatchNormalization(name='norm_2')(x)
          x = LeakyReLU(alpha=0.1)(x)
          x = MaxPooling2D(pool_Size=(2, 2))(x)

          #Layer 3
          x = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_3', use_bias=False)(x)
          x = BatchNormalization(name='norm_3')(x)
          x = LeakyReLU(alpha=0.1)(x)

          #Layer 4
          x = Conv2D(64, (1,1), strides=(1,1), padding='same', name='conv_4', use_bias=False)(x)
          x = BatchNormalization(name='norm_4')(x)
          x = LeakyReLU(alpha=0.1)(x)

          #Layer 5
          x = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_5', use_bias=False)(x)
          x = BatchNormalization(name='norm_5')(x)
          x = LeakyReLU(alpha=0.1)(x)
          x = MaxPooling2D(pool_Size=(2, 2))(x)

          #Layer 6
          x = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_6', use_bias=False)(x)
          x = BatchNormalization(name='norm_6')(x)
          x = LeakyReLU(alpha=0.1)(x)

          #Layer 7
          x = Conv2D(128, (1,1), strides=(1,1), padding='same', name='conv_7', use_bias=False)(x)
          x = BatchNormalization(name='norm_7')(x)
          x = LeakyReLU(alpha=0.1)(x)

          #Layer 8
          x = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_8', use_bias=False)(x)
          x = BatchNormalization(name='norm_8')(x)
          x = LeakyReLU(alpha=0.1)(x)
          x = MaxPooling2D(pool_Size=(2, 2))(x)

          #Layer 9
          x = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_9', use_bias=False)(x)
          x = BatchNormalization(name='norm_9')(x)
          x = LeakyReLU(alpha=0.1)(x)

          #Layer 10
          x = Conv2D(256, (1,1), strides=(1,1), padding='same', name='conv_10', use_bias=False)(x)
          x = BatchNormalization(name='norm_10')(x)
          x = LeakyReLU(alpha=0.1)(x)

          #Layer 11
          x = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_11', use_bias=False)(x)
          x = BatchNormalization(name='norm_11')(x)
          x = LeakyReLU(alpha=0.1)(x)

          #Layer 12
          x = Conv2D(256, (1,1), strides=(1,1), padding='same', name='conv_12', use_bias=False)(x)
          x = BatchNormalization(name='norm_12')(x)
          x = LeakyReLU(alpha=0.1)(x)

          #Layer 13
          x = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_13', use_bias=False)(x)
          x = BatchNormalization(name='norm_13')(x)
          x = LeakyReLU(alpha=0.1)(x)

          skip_connection = x

          x = MaxPooling2D(pool_size=(2, 2))(x)

          #Layer 14
          x = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_14', use_bias=False)(x)
          x = BatchNormalization(name='norm_14')(x)
          x = LeakyReLU(alpha=0.1)(x)

          #Layer 15
          x = Conv2D(512, (1,1), strides=(1,1), padding='same', name='conv_15', use_bias=False)(x)
          x = BatchNormalization(name='norm_15')(x)
          x = LeakyReLU(alpha=0.1)(x)

          #Layer 16
          x = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_16', use_bias=False)(x)
          x = BatchNormalization(name='norm_16')(x)
          x = LeakyReLU(alpha=0.1)(x)

          #Layer 17
          x = Conv2D(512, (1,1), strides=(1,1), padding='same', name='conv_17', use_bias=False)(x)
          x = BatchNormalization(name='norm_17')(x)
          x = LeakyReLU(alpha=0.1)(x)

          #Layer 18
          x = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_18', use_bias=False)(x)
          x = BatchNormalization(name='norm_18')(x)
          x = LeakyReLU(alpha=0.1)(x)

          #Layer 19
          x = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_19', use_bias=False)(x)
          x = BatchNormalization(name='norm_19')(x)
          x = LeakyReLU(alpha=0.1)(x)

          #Layer 20
          x = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_20', use_bias=False)(x)
          x = BatchNormalization(name='norm_20')(x)
          x = LeakyReLU(alpha=0.1)(x)

          #Layer 21
          skip_connection = Conv2D(64, (1,1), strides=(1,1), padding='same', name='conv_21', use_bias=False)(skip_connection)
          skip_connection = BatchNormalization(name='norm_21')(skip_connection)
          skip_connection = LeakyReLU(alpha=0.1)(skip_connection)
          skip_connection = Lambda(space_to_depth_x2)(skip_connection)

          x = concatenate([skip_connection, x])

          #Layer 22
          x = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_22', use_bias=False)(x)
          x = BatchNormalization(name='norm_22')(x)
          x = LeakyReLU(alpha=0.1)(x)

          self.feature_extractor = Model(input_image, x)
          self.feature_extractor.load_weights(FULL_YOLO_BACKEND_PATH)

        def normalize(self, image):
          return image / 255.


#funciones que necesitaremos
def _sigmoid(x):
  return 1./(1. + np.exp(-x))

def _softmax(x, axis=-1, t=-100.):
  x = x-np.max(x)

  if np.min(x) < t:
    x = x/np.min(x)*t

  e_x = np.exp(x)

  return e_x / e_x.sum(axis, keepdims=True)

def _interval_overlap(interval_a, interval_b):
  x1, x2 = interval_a
  x3, x4 = interval_b

  if x3 < x1:
    if x4< x1:
      return 0
    else:
      return min(x2,x4) - x1
  else:
    if x2 < x3:
      return 0
    else:
      return min(x2,x4) -x3

def compute_overlap(a,b):

  area = (b[:,2] - b[:, 0]) * (b[:,3] - b[:,1])

  iw = np.minimum(np.expand_dims(a[:,2], axis=1), b[:,2]) - np.maximum(np.expand_dims(a[:,0], 1), b[:,0])
  ih = np.minimum(np.expand_dims(a[:,3], axis=1), b[:,3]) - np.maximum(np.expand_dims(a[:,1], 1), b[:, 1])

  iw = np.maximum(iw, 0)
  ih = np.maximum(ih, 0)

  ua = np.expand_dims((a[:,2] - a[:, 0]) * (a[:,3] - a[:,1]), axis=1) + area - iw * ih

  ua = np.maximum(ua, np.finfo(float).eps)

  intersection = iw * ih

  return intersection / ua

def computer_ap(recall, precision):
  #first append sentinel values at the end
  mrec = np.concatenate(([0.], recall, [1.]))
  mpre = np.concatenate(([0.], precision, [0.]))

  #compute the precision envelope
  for i in range(mpre.size - 1, 0, -1):
    mpre[i - 1]= np.maximum(mpre[i -1], mpre[i])

  #to calculate area under PR curve, look for points
  #where X axis (recall) changes value
  i = np.where(mrec[1:] != mrec[:-1])[0]

  #and sum (\\Delta recall) * prec
  ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
  return ap

def decode_netout(netout, anchors, nb_class, obj_threshold=0.3, nms_threshold=0.3):
  grid_h, grid_w, nb_box = netout.shape[:3]

  boxes = []

  #decode the output by the network
  netout[...,4] = _sigmoid(netout[...,4])
  netout[...,5:] = netout[...,4][..., np.newaxis] * _softmax(netout[...,5:])
  netout[...,5:] *= netout[..., 5:] > obj_threshold
  for row in range(grid_h):
    for col in range(grid_w):
      for b in range(nb_box):
        #from 4th element onwards are confidence and class classes
        classes = netout[row, col,b,5:]

        if np.sum(classes) > 0:
          #first 4 elements are x,y,w and h
          x, y, w, h = netout[row, col, b, 4:]

          x = (col + _sigmoid(x)) / grid_w #center position, unit : image width
          y = (row + _sigmoid(y)) / grid_h # center position, unit: image height
          w = anchors[2 * b + 0] * np.exp(w) / grid_w #unit: image width
          h = anchors[2 * b + 1] * np.exp(h) / grid_h #unit: image height
          confidence =  netout[row, col, b,4]
          box = BoundBox(x-w/2, y-h/2, x+w/2, y+h/2, confidence, classes)

          boxes.append(box)

  for c in range(nb_class):
    sorted_indices = list(reversed(np.argsort([box.classes[c] for box in boxes])))

    for i in range(len(sorted_indices)):
      index_i = sorted_indices[i]

      if boxes[index_i]. classes[c] == 0:
        continue
      else:
        for j in range(i+1, len(sorted_indices)):
          index_j = sorted_indices[j]
          if bbox_iou(boxes[index_i], boxes[index_j]) >= nms_threshold:
            boxes[index_j].classes[c] = 0

  #remove the boxes which are less likely than a obj_threshold
  boxes = [box for box in boxes if box.get_score() > obj_threshold]

  return boxes


class YOLO(object):
  def __init__(self, input_size, labels, max_box_per_image, anchors):
    self.input_size = input_size
    self.labels = list(labels)
    self.nb_class = len(self.labels)
    self.nb_box = len(anchors)//2
    self.class_wt = np.ones(self.nb_class, dtype='float32')
    self.anchors = anchors
    self.max_box_per_image = max_box_per_image

    input_image = Input(shape=(self.input_size, self.input_size, 3))
    self.true_boxes = Input(shape=(1, 1, 1, max_box_per_image, 4))

    self.feature_extractor = FullYoloFeature(self.input_size)

    print(self.feature_extractor.get_out_shape())
    self.grid_h, self.grid_w = self.feature_extractor.get_output_shape()
    features = self.feature_extractor.extract(input_image)


    #make the object detection layer
    output = Conv2D(self.nb_box * (4 + 1 + self.nb_class), (1, 1), strides=(1,1), name='DetectionLayer', kernel_initializer='lecun_normal')(features)
    output = Reshape((self.grid_h, self.grid_w, self.grid_w, self.nb_box, 4 + 1+ self.nb_class))(output)
    output = Lambda(lambda args: args[0])([output, self.true_boxes])

    self.model = Model([input_image, self.true_boxes], output)

    #initialize the wieghts of the detection layer
    layer = self.model.layers[-4]
    weights = layer.get_weights()

    new_kernel = np.random.normal(size=weights[0].shape)/(self.grid_h*self.grid_w)
    new_bias = np.random.normal(size=weights[1].shape)/(self.grid_h * self.grid_w)

    layer.set_weights([new_kernel, new_bias])

    #print a summary of the whole model
    self.model.summary()

  def custom_loss(self, y_true, y_pred):
   mask_shape = tf.shape(y_true)[:4]

   cell_x = tf.to_float(tf.reshape(tf.tile(tf.range(self.grid_w),[self.grid_h]),(1, self.grid_w, 1, 1)))
   cell_y = tf.transpose(cell_x, (0,2,1,3,4))

   cell_grid = tf.tile(tf.concat([cell_x, cell_y], -1), [self.batch_size, 1, 1, self.nb_box, 1])

   coord_mask = tf.zeros(mask_shape)
   conf_mask = tf.zeros(mask_shape)
   class_mask = tf.zeros(mask_shape)

   seen = tf.Variable(0.)
   total_recall = tf.Variable(0.)

   ###Adjust prediction

   ### adjust x and y
   pred_box_xy = tf.sigmoid(y_pred[..., :2]) + cell_grid

   ###adjust w and h
   pred_box_wh = tf.exp(y_pred[..., 2:4]) * np.reshape(self.anchors, [1,1,1,self.nb_box, 2])

   ###adjust confidence
   pred_box_conf = tf.sigmoid(y_pred[..., 4])

   ##adjust class probabilities
   pred_box_class = y_pred[...,5:]

   ###Adjust truth

   ### adjust x and y
   true_box_xy = y_true[..., 0:2]

   ### adjust w and h
   true_box_wh = y_true[..., 2:4]

   ###adjust confidence
   true_wh_half = true_box_wh / 2.
   true_mins = true_box_xy - true_wh_half
   true_maxes = true_box_xy + true_wh_half

   pred_wh_half = true_box_wh / 2.
   pred_mins = pred_box_xy - pred_wh_half
   pred_maxes = pred_box_xy + pred_wh_half

   intersect_mins = tf.maximum(pred_mins, true_mins)
   intersect_maxes = tf.minimum(pred_maxes, true_maxes)
   intersect_wh = tf.maximum(intersect_maxes - intersect_mins, 0.)
   intersect_areas = intersect_wh[..., 0] * intersect_wh[...,1]

   true_areas = true_box_wh[..., 0] * true_box_wh[..., 1]
   pred_areas = pred_box_wh[..., 0] * pred_box_wh[..., 1]

   union_areas = pred_areas + true_areas - intersect_areas
   iou_scores = tf.truediv(intersect_areas, union_areas)

   true_box_conf = iou_scores * y_true[..., 4]

   ### adjust class probabilities
   true_box_class = tf.argmax(y_true[..., 5:], -1)


   ###Determine the masks
   ##coordinate mask
   coord_mask = tf.expand_dims(y_true[...,4], axis= -1) * self.coord_scale

   ###confidence mask: penelize predictors + penelize boxes with low IOU
   #penalize the confidence of the boxes, which have IOU with some ground truth box < 0.6
   true_xy = self.true_boxes[..., 0:2]
   true_wh = self.true_boxes[..., 2:4]

   true_wh_half = true_wh / 2.
   true_mins = true_xy - true_wh_half
   true_maxes = true_xy + true_wh_half

   pred_xy = tf.expand_dims(pred_box_xy, 4)
   pred_wh = tf.expand_dims(pred_box_wh, 4)

   pred_wh_half = pred_wh / 2.
   pred_mins = pred_xy - pred_wh_half
   pred_maxes = pred_xy + pred_wh_half

   intersect_mins = tf.maximum(pred_mins, true_mins)
   intersect_maxes = tf.minimum(pred_maxes, true_maxes)
   intersect_wh = tf.maximum(intersect_maxes - intersect_mins, 0.)
   intersect_areas = intersect_wh[..., 0] * intersect_wh[..., 1]

   true_areas = true_wh[..., 0] * true_wh[..., 1]
   pred_areas = pred_wh[..., 0] * pred_wh[..., 1]

   union_areas = pred_areas + true_areas - intersect_areas
   iou_scores = tf.truediv(intersect_areas, union_areas)

   best_ious = tf.reduce_max(iou_scores, axis=4)
   conf_max = conf_max + tf.to_float(best_ious < 0.6) * (1 - y_true[..., 4]) * self.no_object_scale

   #penalize the confidence of the boxes, which are responsibñe for sorresponding ground truth box
   conf_mask = conf_max + y_true[..., 4] * self.object_scale

   ##class mask: simply the position of the ground truth boxes ( the predictors)
   class_mask = y_true[..., 4] * tf.gather(self.class_wt, true_box_class) * self.class_scale

   ### Warm-up training
   no_boxes_mask = tf.to_float(coord_mask < self.coord_scale/2.)
   seen = tf.assign_add(seen, 1.)

   true_box_xy, true_box_wh, coord_mask = tf.cond(tf.less(seen, self.warmup_batches+1), lambda: [true_box_xy + (0.5 + cell_grid) * no_boxes_mask, true_box_wh + tf.ones_like(true_box_wh) * np.reshape(self.anchors,[1,1,1,self.nb_box,2])*  no_boxes_mask, tf.ones_like(coord_mask)], lambda: [true_box_xy, true_box_wh, coord_mask])

   ###Finalize the loss
   nb_coord_box = tf.reduce_sum(tf.to_float(coord_mask > 0.0))
   nb_conf_box = tf.reduce_sum(tf.to_float(conf_mask > 0.0))
   nb_class_box = tf.reduce_sum(tf.to_float(class_mask > 0.0))

   loss_xy = tf.reduce_sum(tf.square(true_box_xy-pred_box_xy) * coord_mask) / (nb_coord_box + 1e-6) / 2.
   loss_wh = tf.reduce_sum(tf.square(true_box_wh-pred_box_wh) * coord_mask) / (nb_coord_box + 1e-6) / 2.
   loss_conf = tf.reduce_sum(tf.square(true_box_conf-pred_box_conf) * conf_mask) / (nb_conf_box + 1e-6) / 2.
   loss_class= tf.nn.sparse_softmax_cross_entropy_with_logits(labels=true_box_class, logits=pred_box_class)

   loss= tf.cond(tf.less(seen, self.warmup_batches+1), lambda: loss_xy + loss_wh + loss_conf + loss_class + 10, lambda: loss_xy + loss_wh + loss_conf + loss_class)
   if self.debug:
    nb_true_box = tf.reduce_sum(y_true[...,4])
    nb_pred_box = tf.reduce_sum(tf.to_float(true_box_conf > 0.5) * tf.to_float(pred_box_conf > 0.3))

    current_recall = nb_pred_box/(nb_true_box + 1e-6)
    total_recall = tf.assign_add(total_recall, current_recall)

    loss = tf.Print(loss, [loss_xy], message='Loss XY \\t', summarize=1000)
    loss = tf.Print(loss, [loss_wh], message='Loss WH \\t', summarize=1000)
    loss = tf.Print(loss, [loss_conf], message='Loss Conf \\t', summarize=1000)
    loss = tf.Print(loss, [loss_class], message='Loss Class \\t', summarize=1000)
    loss = tf.Print(loss, [loss], message='Total Loss \\t', summarize=1000)
    loss = tf.Print(loss, [current_recall], message='Current Recall \\t', summarize=1000)
    loss = tf.Print(loss, [total_recall/seen], message='Average Recall \\t', summarize=1000)

   return loss

  def load_weights(self, weight_path):
    self.model.load_weights(weight_path)

  def train(self, train_imgs, valid_imgs, train_times, valis_times, nb_epochs, learning_rate, batch_size, warmup_epochs, object_scale, no_object_scale, coord_scale, class_scale, saved_weights_name='best_weights.h5', debug=False):
    self.batch_size = batch_size
    self.object_scale = object_scale
    self.no_object_scale = no_object_scale
    self.coord_scale = coord_scale
    self.class_scale = class_scale
    self.debug = debug
    ##Make train and validation generators
    generator_config = {
        'IMAGE_H' : self.input_size,
        'IMAGE_W' : self.input_size,
        'GRID_H' : self.grid_h,
        'GRID_W' : self.grid_w,
        'BOX' : self.nb_box,
        'LABELS' : self.labels,
        'CLASS' : len(self.labels),
        'ANCHORS' : self.anchors,
        'BATCH_SIZE' : self.batch_size,
        'TRUE_BOX_BUFFER' : self.max_box_per_image,
    }
    train_generator = BatchGenerator(train_imgs, generator_config, norm=self.feature_extractor.normalize)
    valid_generator = BatchGenerator(valid_imgs, generator_config, norm=self.feature_extractor.normalize, jitter=False)
    self.warmup_batches = warmup_epochs * (train_times*len(train_generator) + valid_times*len(valid_generator))

    #Compile the model

    optimizer = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    self.model.compile(loss=self.custom_loss, optimizer=optimizer)

    #Make a few callbacks
    early_stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3, mode='min', verbose=1)
    checkpoint = ModelCheckpoint(saved_weights_name, monitor='val_loss', verbose=1, save_best_only=True, mode='min', period=1)
    tensorboard = TensorBoard(log_dir=os.path.expanduser('~/logs/'), histogram_freq=0, write_graf=True, write_images=False)
    #Start the training process
    self.model.fit_generator(generator=train_generator, steps_per_epoch=len(train_generator)*train_times, epochs=warmup_epochs + nb_epochs, verbose= 2 if debug else 1, validation_data= valid_generator, validation_steps = len(valid_generator) * valid_times, callbacks = [early_stop, checkpoint, tensorboard], workers = 3, max_queue_size = 8)

    #Compute mAP on the validation set
    average_precisions = sel.evaluate(valid_generator)

    #print evaluation
    for label, average_precision in average_precisions.items():
      print(self.labels[label], '{:.4f}'.format(average_precisions))
    print('mAP: {:.4f}'.format(sum(average_precisions.values())/len(average_precisions)))

  def evaluate(self, generator, iou_threshold=0.3, score_threshold=0.3, max_detection=100, save_path=None):
    all_detections =[[None for i in range(generator.num_classes())] for j in range(generator.size())]
    all_annotations = [[None for i in range(generator.num_classes())] for j in range(generator.size())]

    for i in range(generator.size()):
      raw_image = generator.load_image(i)
      raw_height, raw_width, raw_channels = raw_image.shape
      pred_bpxes = self.predict(raw_image)
      score = np.array([box.score for box in pred_boxes])
      pred_labels = np.array([box.label for box in pred_boxes])
      if len(pred_boxes) > 0:
        pred_boxes = np.array([[box.min*raw_witdh, box.ymin*raw_height, box.xmax*raw_width, box.ymax*raw_height, box.score] for box in pred_boxes])
      else:
        pred_boxes = np.array([[]])

      #sort the boxes and the labels according to score
      score_sort = np.argsort(-score)
      pred_labels = pred_labels[score_sort]
      pred_boxes = pred_boxes[score_sort]

      #copy detections to all_detections
      for label in range(generator.num_classes()):
        all_detections[i][label] = pred_boxes[pred_labels == label, :]

      annotations = generator.load_annotation(i)

      #copy detections to all_annotations
      for label in range(generator.num_classes()):
        all_annotations[i][label] = annotations[annotations[:,4] == label, :4]. copy()

    average_precisions = {}

    for label in range(generator.num_classes()):
      false_positives = np.zeros((0,))
      true_positives = np.zeros((0,))
      scores = np.zeros((0,))
      num_annotations = 0.0

      for i in range(generator.size()):
        detections = all_detections[i][label]
        annotations = all_annotations[i][label]
        num_annotations += annotations.shape[0]
        detected_annotations = []

        for d in detections:
          scores = np.append(scores, d[4])
          if annotations.shape[0] == 0:
            false_positives = np.append(false_positives, 1)
            true_positives = np.append(true_positives, 0)
            continue

          overlaps= compute_overlap(np.expand_dims(d, axis=0),annotations)
          assigned_annotation = np.argmax(overlaps, axis=1)
          max_overlap = overlaps[0, assigned_annotation]

          if max_overlap >= iou_threshold and assigned_annotation not in detected_annotations:
            false_positives = np.append(false_positives, 0)
            true_positives = np.append(true_positives, 1)
            detected_annotations.append(assigned_annotation)
          else:
            false_positives = np.append(false_positives, 1)
            true_positives = np.append(true_positives, 0)

      if num_annotations == 0:
        average_precisions[label] = 0
        continue

      indices= np.argsort(-scores)
      false_positives = false_positives[indices]
      true_positives = true_positives[indices]

      false_positives= np.cumsum(false_positives)
      true_positives = np.cumsum(true_positives)

      recall = true_positives / num_annotations
      precision = true_positives / np.maximum(true_positives + false_positives, np.finfo(np.float64).eps)

      average_precision = compute_ap(recall, precision)
      average_precisions[label] = average_precision

    return average_precisions

  def predict(self, image):
    image_h, image_w, _ = image.shape
    image = cv2.resize(image,(self.input_size, self.input_size))
    image = self.feature_extractor.normalize(image)

    input_image = image[:,:,::-1]
    input_image = np.expand_dims(input_image, 0)
    dummy_array = np.zeros((1,1,1,1,self.max_box_per_image, 4))

    netout = self.model.predict([input_image, dummy_array])[0]
    boxes = decode_netout(netout, self.anchors, self.nb_class)

    return boxes

import random
num_anchors = 5
def IOU(ann, centroids):
  w, h = ann
  similarities = []
  for centroid in centroids:
    c_w, c_h = centroid

    if c_w >= w and c_h >=h:
      similarity = w*h/(c_w*c_h)
    elif c_w >= w and c_h <= h:
      similarity = w*c_h /(w*h + (c_w-w)*c_h)
    elif c_w <= w and c_h >= h:
      similarity = c_w*h/(w*h + c_w*(c_h-h))
    else:
      similarity = (c_w*c_h)/(w*h)
    similarities.append(similarity)

  return np.array(similarities)

def avg_IOU(anns, centroids):
  n,d = anns.shape
  sum = 0.
  for i in range(anns.shape[0]):
    sum+=max(IOU(anns[i], centroids))
  return sum

def print_anchors(centroids):
  anchors = centroids.copy()
  widths = anchors[:,0]
  sorted_indices = np.argsort(widths)
  r = "anchors:[ "
  for i in sorted_indices[:-1]:
    r += '%0.2f,%0.2f, ' % (anchors[i, 0], anchors[i, 1])
    r += "] "
    print(r)

def run_kmeans(ann_dims, anchor_num):
  ann_num = ann_dims.shape[0]
  iterations = 0
  prev_assignments = np.ones(ann_num)*(-1)
  iteration = 0
  old_distances = np.zeros((ann_num, anchor_num))

  indices = [random.randrange(ann_dims.shape[0]) for i in range(anchor_num)]
  centroids = ann_dims[indices]
  anchor_dim = ann_dims.shape[1]
  while True:
    destances = []
    iteration += 1
    for i in range(ann_num):
      d = 1 -IOU(ann_dims[i], centroids)
      distances.append(d)
    distances = np.array(distances)
    print("iteration {}: dists = {} ".format(iteration, np.sum(np.abs(old_distances - distances ))))
    assignments = np.argmin(distances, axis=1)
    if (assignments == prev_assignments).all():
      return centroids
    centroid_sums=np.zers((anchor_num, anchor_dim), np.float)
    for i in range(ann_num):
      centroid_sums[assignments[i]]+=ann_dims[i]
    for j in range(anchor_num):
      centroids[j] = centroid_sums[j]/(np.sum(assignments==j) + 1e-6)
    prev_assignments = assignments.copy()
    old_distances = distances.copy()

tamanio = 416
grid_w = tamanio/32
grid_h = tamanio/32

annotation_dims = []
for  image  in algunas_imagenes[:int(0.8*len(algunas_imagenes))]:
  cell_w= image['width']/grid_w
  cell_h = image['height']/grid_h

  for obj in image['object']:
    relative_w = (float(obj['xmax']) - float(obj['xmin']))/cell_w
    relatice_h = (float(obj['ymax']) - float(obj['ymin']))/cell_h
    annotation_dims.append(tuple(map(float, (relative_w, relatice_h))))

annotation_dims = np.array(annotation_dims)
print(relative_w)
#centroids = run_kmeans(annotation_dims, num_anchors)

#print('\\naverage IOU for', num_anchors, 'anchors:', '%0.2f' % avg_IOU(annotation_dims, centroids))
#print_anchors(centroids)

#anchors=[]
#for x in centroids:
#  anchors.append(x[0])
#  anchors.append(x[1])
#anchors

#instaciamos el modelo
#yolo = YOLO(onput_size=tamanio, labels = labels, max_box_per_image = 5, anchors=anchors)


































FileNotFoundError: [Errno 2] No such file or directory: '/anotaciones/documento.xml'